# <img align="left" src="./images/film_strip_vertical.png"     style=" width:40px;  " > Practice lab: Aprendizaje profundo para el filtrado basado en el contenido

En este ejercicio, implementará el filtrado basado en el contenido utilizando una red neuronal para construir un sistema de recomendación de películas.


# Outline
- [ 1 - Packages ](#1)
- [ 2 - Movie ratings dataset ](#2)
- [ 3 - Content-based filtering with a neural network](#3)
  - [ 3.1 Training Data](#3.1)
  - [ 3.2 Preparing the training data](#3.2)
- [ 4 - Neural Network for content-based filtering](#4)
  - [ Exercise 1](#ex01)
- [ 5 - Predictions](#5)
  - [ 5.1 - Predictions for a new user](#5.1)
  - [ 5.2 - Predictions for an existing user.](#5.2)
  - [ 5.3 - Finding Similar Items](#5.3)
    - [ Exercise 2](#ex02)
- [ 6 - Congratulations! ](#6)


<a name="1"></a>
## 1 - Packages <img align="left" src="./images/movie_camera.png"     style=" width:40px;  ">
We will use familiar packages, NumPy, TensorFlow and helpful routines from [scikit-learn](https://scikit-learn.org/stable/). We will also use [tabulate](https://pypi.org/project/tabulate/) to neatly print tables and [Pandas](https://pandas.pydata.org/) to organize tabular data.

In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
from recsysNN_utils import *
pd.set_option("display.precision", 1)

<a name="2"></a>
## 2 - Movie ratings dataset <img align="left" src="./images/film_rating.png" style=" width:40px;" >
El conjunto de datos se deriva del conjunto de datos [MovieLens ml-latest-small](https://grouplens.org/datasets/movielens/latest/). 

[F. Maxwell Harper y Joseph A. Konstan. 2015. Los conjuntos de datos de MovieLens: Historia y contexto. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1-19:19. <https://doi.org/10.1145/2827872>]

El conjunto de datos original tiene aproximadamente 9000 películas calificadas por 600 usuarios con calificaciones en una escala de 0,5 a 5 en incrementos de 0,5 pasos. Se ha reducido el tamaño del conjunto de datos para centrarse en las películas de los años posteriores al 2000 y en los géneros populares. El conjunto de datos reducido tiene $n_u = 397$ usuarios, $n_m= 847$ películas y 25521 valoraciones. Para cada película, el conjunto de datos proporciona el título, la fecha de estreno y uno o más géneros. Por ejemplo, "Toy Story 3" se estrenó en 2010 y tiene varios géneros: "Aventura|Animación|Infantil|Comedia|Fantasía". Este conjunto de datos contiene poca información sobre los usuarios, aparte de sus valoraciones. Este conjunto de datos se utiliza para crear vectores de entrenamiento para las redes neuronales que se describen a continuación. 
Conozcamos un poco más sobre este conjunto de datos. La siguiente tabla muestra las 10 mejores películas clasificadas por el número de valoraciones. Estas películas también tienen un alto promedio de puntuación. ¿Cuántas de estas películas has visto?

In [2]:
top10_df  = pd.read_csv("./data/content_top10_df.csv")
bygenre_df = pd.read_csv("./data/content_bygenre_df.csv")
top10_df

,movie id,num ratings,ave rating,title,genres
0,4993,198,4.1,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.0,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.1,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.9,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.2,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.8,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.1,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,4.0,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.9,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.2,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


La siguiente tabla muestra la información clasificada por géneros. El número de valoraciones por género varía sustancialmente. Tenga en cuenta que una película puede tener varios géneros, por lo que la suma de las valoraciones que aparecen a continuación es mayor que el número de valoraciones originales.

In [3]:
bygenre_df

,genre,num movies,ave rating/genre,ratings per genre
0,Action,321,3.4,10377
1,Adventure,234,3.4,8785
2,Animation,76,3.6,2588
3,Children,69,3.4,2472
4,Comedy,326,3.4,8911
5,Crime,139,3.5,4671
6,Documentary,13,3.8,280
7,Drama,342,3.6,10201
8,Fantasy,124,3.4,4468
9,Horror,56,3.2,1345


<a name="3"></a>
## 3 - Filtrado basado en el contenido con una red neuronal

En el laboratorio de filtrado colaborativo, usted generó dos vectores, un vector de usuarios y un vector de artículos/películas cuyo producto de puntos predeciría una calificación. Los vectores se obtuvieron únicamente a partir de las valoraciones.   

El filtrado basado en el contenido también genera un vector de características del usuario y de la película, pero reconoce que puede haber otra información disponible sobre el usuario y/o la película que puede mejorar la predicción. La información adicional se proporciona a una red neuronal que genera el vector de usuario y película como se muestra a continuación.
<figure>
    <center> <img src="./images/RecSysNN.png"   style="width:500px;height:280px;" ></center>
</figure>

<a name="3.1"></a>
### 3.1 Datos de entrenamiento
El contenido de la película que se proporciona a la red es una combinación de los datos originales y algunas "características de ingeniería". Recordemos la discusión y el laboratorio de ingeniería de características del curso 1, semana 2, laboratorio 4. Las características originales son el año de estreno de la película y el género de la película presentado como un vector de un solo golpe. Hay 14 géneros. La característica diseñada es una calificación media derivada de las calificaciones de los usuarios. 

El contenido del usuario se compone de características de ingeniería. Se calcula una valoración media por género y por usuario. Además, se dispone de una identificación de usuario, un recuento de valoraciones y una media de valoraciones, pero no se incluyen en el contenido de entrenamiento o predicción. Se llevan con el conjunto de datos porque son útiles para interpretar los datos.

El conjunto de entrenamiento está formado por todas las valoraciones realizadas por los usuarios del conjunto de datos. Algunas valoraciones se repiten para aumentar el número de ejemplos de entrenamiento de géneros poco representados. El conjunto de entrenamiento se divide en dos matrices con el mismo número de entradas, una matriz de usuarios y una matriz de películas/artículos.  

A continuación, cargamos y mostramos algunos de los datos.

In [4]:
# Load Data, set configuration variables
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 50884


Veamos las primeras entradas de la matriz de formación de usuarios.

In [5]:
pprint_train(user_train, user_features, uvs,  u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9
2,22,4.0,4.0,4.2,0.0,0.0,4.0,4.1,4.0,4.0,0.0,3.0,4.0,0.0,3.9,3.9


Algunas de las características del usuario y del artículo/película no se utilizan en el entrenamiento. En la tabla anterior, las características entre paréntesis "[]", como el "ID de usuario", el "recuento de valoraciones" y el "promedio de valoraciones", no se incluyen cuando se entrena y utiliza el modelo.
Arriba puede ver la media de valoraciones por género del usuario 2. Las entradas cero son géneros que el usuario no ha valorado. El vector usuario es el mismo para todas las películas calificadas por un usuario.  
Veamos las primeras entradas de la matriz película/artículo.

In [6]:
pprint_train(item_train, item_features, ivs, i_s, maxcount=5, user=False)

[movie id],year,ave rating,Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
6874,2003,4.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
8798,2004,3.8,1,0,0,0,0,1,0,1,0,0,0,0,0,1
46970,2006,3.2,1,0,0,0,1,0,0,0,0,0,0,0,0,0
48516,2006,4.3,0,0,0,0,0,1,0,1,0,0,0,0,0,1
58559,2008,4.2,1,0,0,0,0,1,0,1,0,0,0,0,0,0


Arriba, la matriz de películas contiene el año de estreno de la película, la calificación media y un indicador para cada género potencial. El indicador es uno por cada género que se aplica a la película. El identificador de la película no se utiliza en el entrenamiento, pero es útil para interpretar los datos.

In [7]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  3.5 4.  4.  4.5]


El objetivo, y, es la calificación de la película dada por el usuario. 

Arriba podemos ver que la película 6874 es una película de Acción/Crime/Thriller estrenada en 2003. El usuario 2 califica las películas de acción con un 3,9 de media. Los usuarios de MovieLens dieron a la película una calificación media de 4. 'y' es 4, lo que indica que el usuario 2 también calificó la película 6874 con un 4. Un solo ejemplo de entrenamiento consiste en una fila de las matrices de usuarios y de elementos y una calificación de y_train.

<a name="3.2"></a>
### 3.2 Preparación de los datos de entrenamiento
Recordemos que en el curso 1, semana 2, exploramos el escalado de características como medio para mejorar la convergencia. Escalaremos las características de entrada utilizando el [scikit learn StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Esto se utilizó en el Curso 1, Semana 2, Laboratorio 5.  A continuación, también se muestra la transformación_inversa para producir las entradas originales. Vamos a escalar las calificaciones del objetivo utilizando un Min Max Scaler que escala el objetivo para estar entre -1 y 1. [scikit learn MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [8]:
# scale training data
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


Para permitirnos evaluar los resultados, dividiremos los datos en conjuntos de entrenamiento y de prueba, como se discutió en el Curso 2, Semana 3. Aquí usaremos [sklean train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) para dividir y barajar los datos. Tenga en cuenta que establecer el estado aleatorio inicial al mismo valor asegura que el elemento, el usuario y y se barajan de forma idéntica.

In [9]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


The scaled, shuffled data now has a mean of zero.

In [10]:
pprint_train(user_train, user_features, uvs, u_s, maxcount=5)

[user id],[rating count],[rating ave],Act ion,Adve nture,Anim ation,Chil dren,Com edy,Crime,Docum entary,Drama,Fan tasy,Hor ror,Mys tery,Rom ance,Sci -Fi,Thri ller
1,0,-1.0,-0.8,-0.7,0.1,-0.0,-1.2,-0.4,0.6,-0.5,-0.5,-0.1,-0.6,-0.6,-0.7,-0.7
0,1,-0.7,-0.5,-0.7,-0.1,-0.2,-0.6,-0.2,0.7,-0.5,-0.8,0.1,-0.0,-0.6,-0.5,-0.4
-1,-1,-0.2,0.3,-0.4,0.4,0.5,1.0,0.6,-1.2,-0.3,-0.6,-2.3,-0.1,0.0,0.4,-0.0
0,-1,0.6,0.5,0.5,0.2,0.6,-0.1,0.5,-1.2,0.9,1.2,-2.3,-0.1,0.0,0.2,0.3
-1,0,0.7,0.6,0.5,0.3,0.5,0.4,0.6,1.0,0.6,0.3,0.8,0.8,0.4,0.7,0.7


<a name="4"></a>
## 4 - Red neuronal para el filtrado basado en el contenido
Ahora, construyamos una red neuronal como la descrita en la figura anterior. Tendrá dos redes que se combinan mediante un producto punto. Construirá las dos redes. En este ejemplo, serán idénticas. Tenga en cuenta que no es necesario que estas redes sean iguales. Si el contenido del usuario fuera sustancialmente mayor que el contenido de la película, podría optar por aumentar la complejidad de la red del usuario en relación con la red de la película. En este caso, el contenido es similar, por lo que las redes son las mismas.

<a name="ex01"></a>
### Exercise 1

- Utilizar un modelo secuencial Keras
    - La primera capa es una capa densa con 256 unidades y una activación relu.
    - La segunda capa es una capa densa con 128 unidades y una activación relu.
    - La tercera capa es una capa densa con `num_outputs` unidades y una activación lineal o no.   
    
El resto de la red se proporcionará. El código proporcionado no utiliza el modelo secuencial de Keras, sino que utiliza la [api funcional] de Keras (https://keras.io/guides/functional_api/). Este formato permite una mayor flexibilidad en la forma de interconectar los componentes.


In [11]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
  
  
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128 ,activation='relu'),
    tf.keras.layers.Dense(num_outputs)
  
  
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu         = user_NN(input_user)
vu         = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm         = item_NN(input_item)
vm         = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 14)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 32)           40864       input_1[0][0]                    
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 32)           41376       input_2[0][0]                    
______________________________________________________________________________________________

In [12]:
# Public tests
from public_tests import *
test_tower(user_NN)
test_tower(item_NN)

All tests passed!
All tests passed!


<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
  You can create a dense layer with a relu activation as shown.
    
```python     
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),

    
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),

    
    ### END CODE HERE ###  
])
```    
<details>
    <summary><font size="2" color="darkblue"><b> Click for solution</b></font></summary>
    
```python 
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])
```
</details>
</details>

    


We will use a mean squared error loss and an Adam optimizer.

In [13]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [14]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Train on 40707 samples
Epoch 1/30
40707/40707 [==============================] - 5s 125us/sample - loss: 0.1232
Epoch 2/30
40707/40707 [==============================] - 5s 118us/sample - loss: 0.1146
Epoch 3/30
40707/40707 [==============================] - 5s 118us/sample - loss: 0.1089
Epoch 4/30
40707/40707 [==============================] - 5s 118us/sample - loss: 0.1039
Epoch 5/30
40707/40707 [==============================] - 5s 117us/sample - loss: 0.1001
Epoch 6/30
40707/40707 [==============================] - 5s 118us/sample - loss: 0.0973
Epoch 7/30
40707/40707 [==============================] - 5s 116us/sample - loss: 0.0956
Epoch 8/30
40707/40707 [==============================] - 5s 116us/sample - loss: 0.0935
Epoch 9/30
40707/40707 [==============================] - 5s 117us/sample - loss: 0.0916
Epoch 10/30
40707/40707 [==============================] - 5s 116us/sample - loss: 0.0897
Epoch 11/30
40707/40707 [==============================] - 5s 117us/sample - loss: 0.0

Evaluar el modelo para determinar la pérdida en los datos de la prueba. 

In [15]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

10177/10177 [==============================] - 0s 35us/sample - loss: 0.0815


0.08146006993124337

Es comparable a la pérdida de entrenamiento, lo que indica que el modelo no ha sobreajustado sustancialmente los datos de entrenamiento.

<a name="5"></a>
## 5 - Predicciones
A continuación, utilizarás tu modelo para hacer predicciones en una serie de circunstancias. 
<a name="5.1"></a>
### 5.1 - Predicciones para un nuevo usuario
Primero, crearemos un nuevo usuario y haremos que el modelo sugiera películas para ese usuario. Después de haber probado esto en el contenido del usuario de ejemplo, siéntase libre de cambiar el contenido del usuario para que coincida con sus propias preferencias y ver lo que el modelo sugiere. Tenga en cuenta que las calificaciones están entre 0,5 y 5,0, inclusive, en incrementos de medio paso.

In [16]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 0.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

Al nuevo usuario le gustan las películas de los géneros de aventura y fantasía. Busquemos las películas mejor valoradas por el nuevo usuario.  
A continuación, utilizaremos un conjunto de vectores de películas/artículos, `item_vecs` que tiene un vector para cada película del conjunto de entrenamiento/prueba. Esto se compara con el vector del nuevo usuario y los vectores escalados se utilizan para predecir las calificaciones de todas las películas.

In [17]:
# generate and replicate the user vector to match the number movies in the data set.
user_vecs = gen_user_vecs(user_vec,len(item_vecs))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display

print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

y_p,movie id,rating ave,title,genres
4.5,98809,3.8,"Hobbit: An Unexpected Journey, The (2012)",Adventure|Fantasy
4.4,8368,3.9,Harry Potter and the Prisoner of Azkaban (2004),Adventure|Fantasy
4.4,54001,3.9,Harry Potter and the Order of the Phoenix (2007),Adventure|Drama|Fantasy
4.3,40815,3.8,Harry Potter and the Goblet of Fire (2005),Adventure|Fantasy|Thriller
4.3,106489,3.6,"Hobbit: The Desolation of Smaug, The (2013)",Adventure|Fantasy
4.3,81834,4,Harry Potter and the Deathly Hallows: Part 1 (2010),Action|Adventure|Fantasy
4.3,59387,4,"Fall, The (2006)",Adventure|Drama|Fantasy
4.3,5952,4,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
4.3,5816,3.6,Harry Potter and the Chamber of Secrets (2002),Adventure|Fantasy
4.3,54259,3.6,Stardust (2007),Adventure|Comedy|Fantasy|Romance


<a name="5.2"></a>
### 5.2 - Predicciones para un usuario existente.
Veamos las predicciones para el "usuario 2", uno de los usuarios del conjunto de datos. Podemos comparar las valoraciones predichas con las valoraciones del modelo.

In [18]:
uid = 2 
# form a set of user vectors. This is the same vector, transformed and repeated.
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]  #using unscaled vectors for display
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]

#print sorted predictions for movies rated by the user
print_existing_user(sorted_ypu, sorted_y.reshape(-1,1), sorted_user, sorted_items, ivs, uvs, movie_dict, maxcount = 50)

y_p,y,user,user genre ave,movie rating ave,movie id,title,genres
4.5,5.0,2,[4.0],4.3,80906,Inside Job (2010),Documentary
4.2,3.5,2,"[4.0,4.0]",3.9,99114,Django Unchained (2012),Action|Drama
4.1,4.5,2,"[4.0,4.0]",4.1,68157,Inglourious Basterds (2009),Action|Drama
4.1,3.5,2,"[4.0,3.9,3.9]",3.9,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller
4.0,4.0,2,"[4.0,4.1,4.0,4.0,3.9,3.9]",4.1,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller
4.0,4.0,2,"[4.1,4.0,3.9]",4.3,48516,"Departed, The (2006)",Crime|Drama|Thriller
4.0,4.5,2,"[4.0,4.1,4.0]",4.2,58559,"Dark Knight, The (2008)",Action|Crime|Drama
4.0,4.0,2,"[4.0,4.1,3.9]",4.0,6874,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller
4.0,3.5,2,"[4.0,4.1,4.0,3.9]",3.8,8798,Collateral (2004),Action|Crime|Drama|Thriller
3.9,5.0,2,"[4.0,4.1,4.0]",3.9,106782,"Wolf of Wall Street, The (2013)",Comedy|Crime|Drama


La predicción del modelo suele estar a menos de 1 de la calificación real, aunque no es un indicador muy preciso de cómo califica un usuario determinadas películas. Esto es especialmente cierto si la valoración del usuario es significativamente diferente de la media del género del usuario. Puede variar la identificación del usuario anterior para probar con diferentes usuarios. No se utilizaron todos los identificadores de usuario en el conjunto de entrenamiento.

<a name="5.3"></a>
### 5.3 - Encontrar elementos similares
La red neuronal anterior produce dos vectores de características, un vector de características del usuario $v_u$, y un vector de características de la película, $v_m$. Se trata de vectores de 32 entradas cuyos valores son difíciles de interpretar. Sin embargo, los artículos similares tendrán vectores similares. Esta información puede utilizarse para hacer recomendaciones. Por ejemplo, si un usuario ha valorado muy bien "Toy Story 3", se podrían recomendar películas similares seleccionando películas con vectores de características similares.

Una medida de similitud es la distancia al cuadrado entre los dos vectores $ \mathbf{v_m^{(k)}}$ and $\mathbf{v_m^{(i)}}$ :
$$\left\Vert \mathbf{v_m^{(k)}} - \mathbf{v_m^{(i)}}  \right\Vert^2 = \sum_{l=1}^{n}(v_{m_l}^{(k)} - v_{m_l}^{(i)})^2\tag{1}$$

<a name="ex02"></a>
### Ejercicio 2

Escribe una función para calcular la distancia al cuadrado.

In [19]:
# GRADED_FUNCTION: sq_dist
# UNQ_C2
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    ### START CODE HERE ###     
    d = np.sum((np.square(np.subtract(b,a))))
    ### END CODE HERE ###     
    return d

In [20]:
a1 = np.array([1.0, 2.0, 3.0]); b1 = np.array([1.0, 2.0, 3.0])
a2 = np.array([1.1, 2.1, 3.1]); b2 = np.array([1.0, 2.0, 3.0])
a3 = np.array([0, 1, 0]);       b3 = np.array([1, 0, 0])
print(f"squared distance between a1 and b1: {sq_dist(a1, b1):0.3f}")
print(f"squared distance between a2 and b2: {sq_dist(a2, b2):0.3f}")
print(f"squared distance between a3 and b3: {sq_dist(a3, b3):0.3f}")

squared distance between a1 and b1: 0.000
squared distance between a2 and b2: 0.030
squared distance between a3 and b3: 2.000


**Expected Output**:

squared distance between a1 and b1: 0.000    
squared distance between a2 and b2: 0.030   
squared distance between a3 and b3: 2.000

In [21]:
# Public tests
test_sq_dist(sq_dist)

All tests passed!


<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
  While a summation is often an indication a for loop should be used, here the subtraction can be element-wise in one statement. Further, you can utilized np.square to square, element-wise, the result of the subtraction. np.sum can be used to sum the squared elements.
    
</details>

    


Una matriz de distancias entre películas puede calcularse una vez cuando se entrena el modelo y luego reutilizarse para nuevas recomendaciones sin necesidad de volver a entrenar. El primer paso, una vez entrenado el modelo, es obtener el vector de características de la película, $v_m$, para cada una de las películas. Para ello, utilizaremos la `item_NN` entrenada y construiremos un pequeño modelo que nos permita pasar los vectores de las películas por él para generar $v_m$.

In [22]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 16)]         0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 32)           41376       input_3[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_l2_normalize_2/Squa [(None, 32)]         0           sequential_1[1][0]               
__________________________________________________________________________________________________
tf_op_layer_l2_normalize_2/Sum  [(None, 1)]          0           tf_op_layer_l2_normalize_2/Square
____________________________________________________________________________________________

Una vez que tenga un modelo de película, puede crear un conjunto de vectores de características de la película utilizando el modelo para predecir utilizando un conjunto de vectores de elementos/películas como entrada. El conjunto `item_vecs` es un conjunto de todos los vectores de la película. Debe ser escalado para utilizarlo con el modelo entrenado. El resultado de la predicción es un vector de características de 32 entradas para cada película.

In [23]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

size of all predicted movie feature vectors: (847, 32)


Ahora vamos a calcular una matriz de la distancia al cuadrado entre cada vector de características de la película y todos los demás vectores de características de la película:
<figure>
    <left> <img src="./images/distmatrix.PNG"   style="width:400px;height:225px;" ></center>
</figure>

We can then find the closest movie by finding the minimum along each row. We will make use of [numpy masked arrays](https://numpy.org/doc/1.21/user/tutorial-ma.html) to avoid selecting the same movie. The masked values along the diagonal won't be included in the computation.

In [ ]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )
table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
table

The results show the model will generally suggest a movie with similar genre's.

<a name="6"></a>
## 6 - Congratulations! <img align="left" src="./images/film_award.png" style=" width:40px;">
You have completed a content-based recommender system.    

This structure is the basis of many commercial recommender systems. The user content can be greatly expanded to incorporate more information about the user if it is available.  Items are not limited to movies. This can be used to recommend any item, books, cars or items that are similar to an item in your 'shopping cart'.